In [10]:
from deta import Deta
import pandas as pd
import json
from pandas import json_normalize
from collections import Counter
import plotly.graph_objects as go               
import plotly.express as px                     
from plotly.subplots import make_subplots
import streamlit_authenticator as stauth

In [11]:
DETA_KEY = "c0jo61nr_BhSm5qHprUP75vRSdEmumYfoS1KMCtQW"

# Initialize with a project key
deta = Deta(DETA_KEY)

# This is how to create/connect a database
db = deta.Base("test_db")

In [12]:
def insert_data(username, name, email,address,password,hashed_password,ic,remark):
    """Returns the user on a successful user creation, otherwise raises and error"""
    return db.put({"key": username, "name": name, "email": email, "address": address, "password": password,
                   "hashed_password": hashed_password, "ic": ic, "remarks": remark})

def fetch_all_data():
    """Returns a dict of all periods"""
    res = db.fetch()
    return res.items

def get_data(data):
    """If not found, the function will return None"""
    return db.get(data)

def update_data(username, updates):
    """If the item is updated, returns None. Otherwise, an exception is raised"""
    return db.update(updates, username)

def delete_user(username):
    """Always returns None, even if the key does not exist"""
    return db.delete(username)

In [5]:
r = db.get('zzahir1978').values()
r

dict_values(['Bandar Mahkota Cheras', 'zahiruddin.zahidanishah@gamil.com', 'A3938876', 'zzahir1978', 'Zahiruddin Zahidanishah', 'abc123', 'Test Data'])

In [46]:
# Insert Data
username = 'zzahir1978'
name = 'Zahiruddin Zahidanishah'
email = 'zzahir1978@gmail.com'
address = 'Bandar Mahkota Cheras'
password = 'abc123'
ic = '780113086999'
remark = 'Test Data #1'
hashed_password = stauth.Hasher(password).generate()
insert_data(username, name, email,address,password,hashed_password,ic,remark)

{'address': 'Bandar Mahkota Cheras',
 'email': 'zzahir1978@gmail.com',
 'hashed_password': ['$2b$12$gU7t3bJwY73JtLiUIAXy1uqsXd.GNPNOXwYc5wtwYbZ6AupNnokwO',
  '$2b$12$uc06k.Z/MKk3fqy.wlzTP.BpplQ3H3su0UOibHgdhxYPXmezWv0VS',
  '$2b$12$Ivaljr4DJSU8NxEbQuhXJuOy/oiRszzPkGtIwoI7p1YnLV45223MK',
  '$2b$12$9aTGVjS3ngfF8uxN8N2Sxea.aI00jIcMDOzqpty2GIjlDysDn3lUS',
  '$2b$12$VPIAcBm6e.x6qyg0/3cUo.U1CDLDvCVc3ExlG/kGdz2PBY19Dj3FK',
  '$2b$12$SZqEDRnB.yi1tpays3p30uiC5FX/DQKlgCwQAIgf8ZQUrb54JOTMm'],
 'ic': '780113086999',
 'key': 'zzahir1978',
 'name': 'Zahiruddin Zahidanishah',
 'password': 'abc123',
 'remarks': 'Test Data #1'}

In [47]:
# Update Data
username = 'zzahir1978'
name = 'name'
nam = 'Zahiruddin Bin Zahidanishah'
email = 'email'
ema = 'zahiruddin.zahidanishah@gmail.com'
address = 'address'
add = 'BMC Selangor'
password = 'password'
pas = 'def456'
ic = 'ic'
ics = "780113086999"
remark = 'remarks'
rem = 'Test Data #2'
hashed_password = 'hashed_password'
has = stauth.Hasher(password).generate()
#hashed_password = stauth.Hasher(password).generate()
#updates = {name, email,address,password,hashed_password,ic,remark}
update_data(username, updates={name:nam, email:ema, address:add, password:pas, ic:ics, remark:rem, 
                               hashed_password:has})